## Description
This notebook produces submission with the driver state predictions and the hazard tracks predictions.
- The driver state predictions are based on change of total amount of bounding box pixels between frames of videos.
- The hazard track predictions are based on extension of the baseline approach where all available tracks for each frame are predicted as hazard tracks. Hazard track IDs are sorted based on the proximity to the center of the video frame.
- Results are saved at ```submissions/```

In [ ]:
import pandas as pd
import os
import os.path as osp

from tqdm import tqdm
from utils.ioutils import load_pickle
from utils.process_video import video_results_to_submission_format, process_video

%load_ext autoreload
%autoreload 2

In [5]:
ANNOTATIONS_PATH = "resources/annotations_public.pkl"
annotations = load_pickle(ANNOTATIONS_PATH)
VIDEOS_ROOT = "dataset/coool-benchmark"                       # <---- UPDATE THIS ONE

for video_name in annotations.keys():#Check that videos exist
    assert osp.exists(os.path.join(VIDEOS_ROOT, f"{video_name}.mp4"))

In [7]:
submission_results = {}

video_names = sorted(list(annotations.keys()))
for video_name in tqdm(video_names, total=len(video_names)):
    video_path = osp.join(VIDEOS_ROOT, f"{video_name}.mp4")
    video_results = process_video(video_path, annotations, run_tracks=True, run_captions=False)
    _submission_results = video_results_to_submission_format(video_results)
    submission_results.update(_submission_results)

df = pd.DataFrame.from_dict(submission_results, orient="index")
df.index.name = "ID"

100%|██████████| 200/200 [02:52<00:00,  1.16it/s]


In [8]:
remove_cols = ["Hazard_Track_ ", "Hazard_Name_ "]
df.drop(remove_cols, axis="columns", inplace=True, errors="ignore")
df

,Driver_State_Changed,Hazard_Track_0,Hazard_Name_0,Hazard_Track_1,Hazard_Name_1,Hazard_Track_2,Hazard_Name_2,Hazard_Track_3,Hazard_Name_3,Hazard_Track_4,...,Hazard_Track_18,Hazard_Name_18,Hazard_Track_19,Hazard_Name_19,Hazard_Track_20,Hazard_Name_20,Hazard_Track_21,Hazard_Name_21,Hazard_Track_22,Hazard_Name_22
ID,,,,,,,,,,,,,,,,,,,,,
video_0001_0,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_1,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_2,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_3,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_4,False,2,,3,,4,,5,,6,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
video_0200_365,True,1,,0,,,,,,,...,,,,,,,,,,
video_0200_366,True,1,,0,,,,,,,...,,,,,,,,,,
video_0200_367,True,1,,0,,,,,,,...,,,,,,,,,,


In [10]:
df.to_csv("submissions/results_bboxsizes_driver_alltracks.csv", sep=",")